In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

In [2]:
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.optim import AdamW

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
SOS_token = 0
EOS_token = 1

MAX_LENGTH = 10
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 
    # and p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs



input_lang, output_lang, pairs = prepareData('spa', 'eng', False)
print(random.choice(pairs))


Reading lines...
Read 141370 sentence pairs
Trimmed to 119484 sentence pairs
Counting words...
Counted words:
spa 12091
eng 23395
['don t eat between meals', 'no comas entre comidas', 'cc by france attribution tatoeba org ck hundo']


In [4]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('spa', 'eng', False)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt,_) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

batch_size = 256
input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

Reading lines...
Read 141370 sentence pairs
Trimmed to 119484 sentence pairs
Counting words...
Counted words:
spa 12091
eng 23395


In [5]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        output, hidden = self.gru(embedded)
        return output, hidden

In [6]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        # decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, x, hidden):
        output = self.embedding(x)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [22]:
mask = torch.ones(n_embd,n_embd)
mask = mask.masked_fill(~torch.tril(mask).to(bool), -torch.inf)

tensor([[1., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., -inf, -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., 1., -inf, -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., 1., 1., -inf, -inf, -inf, -inf],
        [1., 1., 1., 1., 1., 1., 1., -inf, -inf, -inf],
        [1., 1., 1., 1., 1., 1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [27]:
import math 

class Attention(nn.Module):
    def __init__(self, n_embd):
        self.n_embd = n_embd
        self.wq = nn.Linear(n_embd, n_embd)
        self.wk = nn.Linear(n_embd, n_embd)
        self.wv = nn.Linear(n_embd, n_embd)
        causal_mask = torch.ones(n_embd,n_embd)
        causal_mask = mask.masked_fill(~torch.tril(mask).to(bool), -torch.inf)
        torch.register_buffer('causal_mask', causal_mask)
    
    def forward(self,q,k,v,mask,causal:bool): 
        # q: B,T1,C
        # k: B,T2,C
        # v: B,T2,C
        q = self.wq(q) # B,T1,C
        k = self.wk(k) # B,T2,C
        v = self.wv(v) # B,T2,C
        
        weights = q@k.transpose(-1,-2) / math.sqrt(self.n_embd) # B,T1,T2
        if causal:
            weights *= self.causal_mask
        attn = torch.softmax(weights) # B,T1,T2
        x = attn @ v # B,T1,C
        return x

class Encoder(nn.Module):
    def __init__(self, input_size, n_embd):
        self.embedding = nn.Embedding(input_size, n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.attn = Attention(n_embd)
        
        self.ln2 = nn.LayerNorm(n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.Relu(),
            nn.Linear(n_embd*4, n_embd)
        )

    def forward(self,x):
        o = self.embedding(x)
        o = self.ln1(o)
        o = self.attn(o,o,o,causal=True)
        o = self.mlp(self.ln2(o))
        return o


class Decoder(nn.Module):
    def __init__(self, input_size, n_embd):
        self.embedding = nn.Embedding(input_size, n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.attn = Attention(n_embd)

        self.x_ln1 = nn.LayerNorm(n_embd)
        self.x_attn = Attention(n_embd)
        
        self.ln2 = nn.LayerNorm(n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.Relu(),
            nn.Linear(n_embd*4, n_embd)
        )

    def forward(self,x,decoder_outputs):
        o = self.embedding(x)
        o = self.ln1(query=o,key=decoder_outputs,value=decoder_outputs,causal=)
        o = self.attn()
        o = self.mlp(self.ln2(o))
        return o    
        
    # def forward(self):

In [12]:
device = torch.device('cuda')
input_size = input_lang.n_words
# hidden_size = 64
n_embd = 32
dropout=0.0
output_size = output_lang.n_words
encoder = Encoder(input_size, n_embd).to(device)
decoder = DecoderRNN(n_embd, output_size).to(device)

lr = 1e-2
optim1 = AdamW(params=encoder.parameters(),lr=lr)
optim2 = AdamW(params=decoder.parameters(),lr=lr)

In [28]:
enc_outputs.shape

torch.Size([256, 10, 64])

In [13]:
step = 0
for epoch in range(10):
    for X,y in train_dataloader:
        step +=1
        X = X.to(device)
        y = y.to(device)
        enc_outputs, enc_hidden = encoder(X)
        decoder_outputs, decoder_hidden, _ = decoder(enc_outputs, enc_hidden, target_tensor=y)
        loss = F.cross_entropy(decoder_outputs.view(-1, decoder_outputs.shape[-1]), y.view(-1))
        if step % 100 == 0:
            print(f'e{epoch}:s{step} | loss:{loss:.3f}')
        optim1.zero_grad()
        optim2.zero_grad()
        loss.backward()
        optim1.step()
        optim2.step()
        # break


e0:s100 | loss:3.685
e0:s200 | loss:3.442
e0:s300 | loss:3.131
e0:s400 | loss:3.014
e1:s500 | loss:2.424
e1:s600 | loss:2.297
e1:s700 | loss:2.202
e1:s800 | loss:2.075
e1:s900 | loss:1.969
e2:s1000 | loss:1.621


KeyboardInterrupt: 

In [14]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')
        
evaluateRandomly(encoder,decoder)

> thanks for waiting for me
= gracias por esperarme
< gracias por mi <EOS>

> it was fun doing that
= fue divertido hacerlo
< fue facil que eso lo que eso <EOS>

> tom doesn t seem to understand
= tom parece no entender
< tom no parece que entender <EOS>

> they don t know yet
= ellos todavia no lo saben
< no sabemos no sabe <EOS>

> she visits him twice a year
= ella lo visita dos veces por ano
< ella le dio una semana pasada <EOS>

> you shouldn t see her now
= sera mejor que no te encuentres con ella ahora
< no deberias hablar a nadie ahora <EOS>

> if it breaks it s no big deal
= si se rompe no pasa nada
< si si no se es tan grande <EOS>

> how will you write it ?
= como lo escribiras ?
< como supiste ? <EOS>

> i m going to go there
= voy a ir alli
< voy a ir a ir alla <EOS>

> she danced with joy
= ella bailaba de alegria
< ella empezo a jugar al hospital <EOS>

